# Demo of application using lume-epics and lume-model 

In [ ]:
import numpy as np
from lume_epics.epics_server import Server
from lume_epics.model import SurrogateModel
from lume_model.variables import ScalarInputVariable, ImageOutputVariable
from lume_model.utils import save_variables

## Model class
The model class is as subclass of the lume_epics.model.SurrogateModel class, which will validate that the subclasses have assigned compatible input/output variables and a predict method.

In [ ]:
class DemoModel(SurrogateModel):
    input_variables = {
        "input1": ScalarInputVariable(name="input1", value=1, range=[0, 256]),
        "input2": ScalarInputVariable(name="input2", value=2, range=[0, 256]),
    }

    output_variables = {
        "output1": ImageOutputVariable(
            name="output1", axis_labels=["value_1", "value_2"], axis_units=["mm", "mm"], x_min=0, x_max=50, y_min=0, y_max=50
        )
    }
    
    def evaluate(self, input_variables):
        self.output_variables["output1"].value = np.random.uniform(
            self.input_variables["input1"].value, # lower dist bound
            self.input_variables["input2"].value, # upper dist bound
            (50,50)
        ) #shape
        

        return list(self.output_variables.values())


# What happens if a model class is missing outputs?

In [ ]:
class FailureClass1(SurrogateModel):
    input_variables = {
        "input1": ScalarInputVariable(name="input1", value=1, range=[0, 256]),
        "input2": ScalarInputVariable(name="input2", value=2, range=[0, 256]),
    }

    
    def evaluate(self, input_variables):
        return list(self.input_variables.values())

FailureClass1()

# ... Or an evaluate method:

In [ ]:
class FailureClass2(SurrogateModel):
    input_variables = {
        "input1": ScalarInputVariable(name="input1", value=1, range=[0, 256]),
        "input2": ScalarInputVariable(name="input2", value=2, range=[0, 256]),
    }

    output_variables = {
        "output1": ImageOutputVariable(
            name="output1", axis_labels=["value_1", "value_2"], axis_units=["mm", "mm"], x_min=0, x_max=50, y_min=0, y_max=50
        )
    }
    
    def return_prediction(self, input_variables):
        self.output_variables["output1"].value = np.random.uniform(
            self.input_variables["input1"].value, # lower dist bound
            self.input_variables["input2"].value, # upper dist bound
            (50,50)
        ) #shape
        

        return list(self.output_variables.values())
    
    
FailureClass2()

# Server
Setting up Channel Access and PVAccess servers requires passing only prefix and the model class. Single protocol servers can be configured by passing `protocol=[{PROTOCOL}]`.

In [ ]:
prefix = "test"
server = Server(DemoModel, prefix)
# monitor = False does not loop in main thread
server.start(monitor=False)

# Save variables

In [ ]:
variable_filename = "demo_variables.pickle"
save_variables(DemoModel, variable_filename)

# See DemoClient.ipynb for display

# Stop the server

In [ ]:
server.stop()